In [1]:
##### Library #####
import cv2
import numpy as np
import time
from DobotControl import DobotControl

In [2]:
##### Image Process Function ######

# Main - input(hsv image, normal image) output(image with box,line,text drawing on top; change on _frame)
def Mask_n_Draw(_frame):
    MinContourArea = 500

    _frame_hsv = cv2.cvtColor(_frame, cv2.COLOR_BGR2HSV) # create new(extra) frame(image) with hsv color

    # HSV color boundry list [red_lower, red_upper]
    hsv_red1   = [np.array([ 0, 38, 0]), np.array([ 4, 255, 255])]
    hsv_red2   = [np.array([ 163, 60, 0]), np.array([ 180, 255, 255])]
    hsv_yellow = [np.array([ 21, 50, 0]), np.array([ 40, 255, 255])]
    hsv_green  = [np.array([ 42, 52, 0]), np.array([ 79, 255, 255])]
    hsv_blue   = [np.array([ 83, 95, 0]), np.array([ 127, 255, 255])]
    hsv_purple = [np.array([ 116, 46, 0]), np.array([ 170, 255, 255])]

    # create mask image using hsv color system
    mask_red1   = cv2.inRange(_frame_hsv, hsv_red1[0], hsv_red1[1])
    mask_red2   = cv2.inRange(_frame_hsv, hsv_red2[0], hsv_red2[1])
    mask_yellow = cv2.inRange(_frame_hsv, hsv_yellow[0], hsv_yellow[1])
    mask_green  = cv2.inRange(_frame_hsv, hsv_green[0], hsv_green[1])
    mask_blue   = cv2.inRange(_frame_hsv, hsv_blue[0], hsv_blue[1])
    mask_purple = cv2.inRange(_frame_hsv, hsv_purple[0], hsv_purple[1])

    # merge red1 and red2
    mask_red = cv2.bitwise_or(mask_red1, mask_red2)
    
    # Contour Retrieval Mode
    # https://docs.opencv.org/3.4/d9/d8b/tutorial_py_contours_hierarchy.html
    CV2_retr = cv2.RETR_TREE # cv2.RETR_TREE, cv2.RETR_LIST, cv2.RETR_EXTERNAL
    # Contour Approximation Method
    # https://docs.opencv.org/4.x/d4/d73/tutorial_py_contours_begin.html
    CV2_approx = cv2.CHAIN_APPROX_SIMPLE # cv2.CHAIN_APPROX_NONE, cv2.CHAIN_APPROX_SIMPLE

    # create contor(numpy array of (x,y) coordinate's') for each object form mask image
    contours_red, _ = cv2.findContours(mask_red, CV2_retr, CV2_approx)
    contours_yellow, _ = cv2.findContours(mask_yellow, CV2_retr, CV2_approx)
    contours_green, _ = cv2.findContours(mask_green, CV2_retr, CV2_approx)
    contours_blue, _ = cv2.findContours(mask_blue, CV2_retr, CV2_approx)
    contours_purple, _ = cv2.findContours(mask_purple, CV2_retr, CV2_approx)

    # use contor location and size info to draw box,line,text on image
    Contour_draw_Rbox(contours_red, MinContourArea, (0,0,255), _frame) # draw rotate box
    Contour_draw_Rbox(contours_yellow, MinContourArea, (0,255,255), _frame)
    Contour_draw_Rbox(contours_green, MinContourArea, (0,255,0), _frame)
    Contour_draw_Rbox(contours_blue, MinContourArea, (255,0,0), _frame)
    Contour_draw_Rbox(contours_purple, MinContourArea, (255,0,255), _frame)

    Contour_draw_Rline(contours_red, MinContourArea, (0,0,255), _frame) # draw rotate center line
    Contour_draw_Rline(contours_yellow, MinContourArea, (0,255,255), _frame)
    Contour_draw_Rline(contours_green, MinContourArea, (0,255,0), _frame)
    Contour_draw_Rline(contours_blue, MinContourArea, (255,0,0), _frame)
    Contour_draw_Rline(contours_purple, MinContourArea, (255,0,255), _frame)

    Contour_draw_Text(contours_red, MinContourArea, (0,0,255), 'red', _frame) # draw text
    Contour_draw_Text(contours_yellow, MinContourArea, (0,255,255), 'yellow', _frame)
    Contour_draw_Text(contours_green, MinContourArea, (0,255,0), 'green', _frame)
    Contour_draw_Text(contours_blue, MinContourArea, (255,0,0), 'blue', _frame)
    Contour_draw_Text(contours_purple, MinContourArea, (255,0,255), 'purple', _frame)

    get_object_list(contours_red, MinContourArea, 0) # pass object to global list
    get_object_list(contours_yellow, MinContourArea, 1)
    get_object_list(contours_green, MinContourArea, 2)
    get_object_list(contours_blue, MinContourArea, 3)
    get_object_list(contours_purple, MinContourArea, 4)

# draw rotate box of object on image : https://docs.opencv.org/3.4/dd/d49/tutorial_py_contour_features.html#:~:text=7.b.-,Rotated%20Rectangle,-Here%2C%20bounding%20rectangle
def Contour_draw_Rbox(Contours, min_area, box_color, frame2draw):
    for cnt in Contours: # for each object(contour)
        if cv2.contourArea(cnt) > min_area: # will reject if object area too small
            rect = cv2.minAreaRect(cnt) # center of box(x,y) , size (w,h) , angle
            box = cv2.boxPoints(rect) # get 4 corners of rotated box
            box = np.int32(box) # convert to int to avoid decimal just in case
            cv2.drawContours(frame2draw, [box], 0, box_color, 1) # image, contour, contour index, color, line thickness

# draw rotate line on center of object on image
def Contour_draw_Rline(Contours, min_area, box_color, frame2draw):
    for cnt in Contours:
        if cv2.contourArea(cnt) > min_area:
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.int32(box)
            xy_cen = np.int32(rect[0])
            wing_12 = (int((box[1][0]+box[2][0])/2),int((box[1][1]+box[2][1])/2)) # get left/right side of Rbox
            wing_03 = (int((box[0][0]+box[3][0])/2),int((box[0][1]+box[3][1])/2))
            cv2.circle(frame2draw, xy_cen, 0, box_color, 10) # big dot on center
            cv2.arrowedLine(frame2draw, xy_cen, wing_12, box_color, 1) # draw arrow line from center to side of box
            cv2.arrowedLine(frame2draw, xy_cen, wing_03, box_color, 1)

# draw text on image :
def Contour_draw_Text(Contours, min_area, box_color, text, frame2draw):
    for cnt in Contours:
        if cv2.contourArea(cnt) > min_area:
            x,y,w,h = cv2.boundingRect(cnt)
            rect = cv2.minAreaRect(cnt)
            rotate = rect[2]
            if text != '': # draw text if 'text' argument not empty
                cv2.putText(frame2draw, text+f': {rotate:.1f}', (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, box_color, 2)

# add object location to global level list
def get_object_list(Contours, min_area, object_list_number):
    global object_list_status
    if object_list_status == True:
        global object_list
        for cnt in Contours:
            if cv2.contourArea(cnt) > min_area:
                rect = cv2.minAreaRect(cnt)
                rotate = rect[2]
                xy_cen = np.int32(rect[0])
                object_list[object_list_number].append([xy_cen[0],xy_cen[1], rotate])

In [5]:
##### Main 1 - Realtime View ######
print('>> Phase 1')

object_list_status = False # not collect object to list

# Read Video/ Cam
# cap = cv2.VideoCapture('5LYVENK.mp4')
cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_AUTO_EXPOSURE,0)
cap.set(cv2.CAP_PROP_AUTOFOCUS,0)
cap.set(cv2.CAP_PROP_AUTO_WB,0)
cap.set(cv2.CAP_PROP_EXPOSURE,-100.0)
cap.set(cv2.CAP_PROP_FOCUS,5)
cap.set(cv2.CAP_PROP_BRIGHTNESS,255)
cap.set(cv2.CAP_PROP_CONTRAST,255)
cap.set(cv2.CAP_PROP_SATURATION,255)

# .get image properties https://docs.opencv.org/3.4/d4/d15/group__videoio__flags__base.html#gaeb8dd9c89c10a5c63c139bf7c4f5704d
vid_w = int(cap.get(3)) 
vid_h = int(cap.get(4))
vid_fps = int(cap.get(cv2.CAP_PROP_FPS)) # or use int(cap.get(5))
vid_scale = 1 # image scale; 1=100% 0.5=50%
print(f'video size = {vid_w} x {vid_h}\nframerate = {vid_fps}')

# fix distortion vairable
dist = np.array([[-0.189400, -0.027179, -0.003365, 0.001168, 0.0]])
cameraMatrix = np.array([[878.507989, 0.        , 331.792216],
                         [0.        , 876.710299, 235.020060],
                         [0.        , 0.        , 1.]])
newCameraMatrix, roi = cv2.getOptimalNewCameraMatrix(cameraMatrix, dist, (vid_w,vid_h), 1, (vid_w,vid_h))

while (cap.isOpened()):
    # read and check for image availability before continue
    check, frame = cap.read()
    if check != True:
        print('>> video ended or not available')
        break

    # pre-process image
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix) # undistort
    frame = cv2.resize(frame, (int(vid_w*vid_scale),int(vid_h*vid_scale))) # resize (optional)
    

    # backup for lastest pre-process frame from video
    frame_lastClean = cv2.resize(frame, (int(vid_w*vid_scale),int(vid_h*vid_scale)))
    # put in any cv function to duplicate to separate image
    # if not, any draw on 'frame' will also affect 'frame_lastClean'

    # start drawing
    Mask_n_Draw(frame)

    # show window
    cv2.imshow('RGB', frame)
    
    
    if cv2.waitKey(1) & 0xFF == ord('e'):
        print('>> Keyboard Exit')
        break
cap.release()
cv2.destroyAllWindows()

print('>> Exit Phase 1')

>> Phase 1
video size = 640 x 480
framerate = 30
>> Keyboard Exit
>> Exit Phase 1


In [6]:
##### Main 2 - Still Image Process ######
print('>> Phase 2')

object_list_status = True # enable collect object to list
object_list = [[],[],[],[],[]] # R,Y,G,B,P

frame = frame_lastClean

# use lastest frame from previous Phase to find object and draw
Mask_n_Draw(frame)

# amount of object found
obj_red = len(object_list[0])
obj_yellow = len(object_list[1])
obj_green = len(object_list[2])
obj_blue = len(object_list[3])
obj_purple = len(object_list[4])
print(f'Object count\n   red   : {obj_red:>2}\n   yellow : {obj_yellow:>2}\n   green  : {obj_green:>2}\n   blue  : {obj_blue:>2}\n   purple  : {obj_purple:>2}\n')

print(object_list[0]) # show red location
print(object_list[1]) # Y
print(object_list[2]) # G
print(object_list[3]) # B
print(object_list[4]) # P

# print(object_list) # everything
# print(object_list[0]) # all red
# print(object_list[0][0]) # red 1st
# print(object_list[0][0][0]) # x in red first
# print(object_list[0][0][1]) # y in red first
# print(object_list[0][0][2]) # r in red first

object_list_merge_xy = [] # create simple list for dobot move order

for i in object_list: # each specific color
    if not i: # skip if list contain nothing ([] mean false or 0)
        continue
    else:
        for j in i: # each object in specific color
            k = object_list.index(i) # index for color list
            j_ = [j[0],j[1],k] # new temporary list contain [x,y,color(cup) index]; add j[2] if need angle
            object_list_merge_xy.append(j_) # add [x,y,color index] to end of list

print(f'\n{object_list_merge_xy}')

cv2.imshow('RGB_Still', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

print('>> Exit Phase 2')

>> Phase 2
Object count
   red   :  1
   yellow :  0
   green  :  1
   blue  :  0
   purple  :  1

[[235, 394, 13.392498016357422]]
[]
[[297, 185, 2.2311742305755615]]
[]
[[19, 405, 90.0]]

[[235, 394, 0], [297, 185, 2], [19, 405, 4]]
>> Exit Phase 2


In [ ]:
##### Dobot Function ######

# Initiate Dobot
class Robot(DobotControl):
    def __init__(self):
        super().__init__()

if __name__ == '__main__':
    dobot = Robot()
    dobot.setAddr(Robot.search()[0])
    dobot.startRobot()

# map Dobot and Camera offset,range to 0-100 percent
class C2D: # Camera to Dobot workspace calculator
    def __init__(self):
        self.dobot_min_x =  207.8959 # 'dobot west most' ; xy from M1 dobot external program
        self.dobot_max_x =  368.1212 # 'dobot east most'
        self.dobot_min_y = -105.4796 # 'dobot south most'
        self.dobot_max_y =  120.3360 #'dobot north most'
        self.dobot_min_z =  68 # 'dobot bottom most'
        self.dobot_max_z =  145 # 'dobot upper most'
        self.cam_min_x =  84 # 'cam west most' ; xy from Phase 2
        self.cam_max_x = 515 # 'cam east most'
        self.cam_min_y =  59 # 'cam north most'
        self.cam_max_y = 359 # 'cam south most'
        self.dobot_dif_x = self.dobot_max_x - self.dobot_min_x # dobot WorkSpace size
        self.dobot_dif_y = self.dobot_max_y - self.dobot_min_y
        self.cam_dif_x = self.cam_max_x - self.cam_min_x # cam WorkSpace size
        self.cam_dif_y = self.cam_max_y - self.cam_min_y
        # dobot +X = cam +Y   ; dobot +Y = cam +X
        # dobot -X = cam -Y   ; dobot -Y = cam -X

    def cam2botX(self,camX,camY): # pass cam x,y pixel to get dobot x coordinate
        camY = (camY - self.cam_min_y) / self.cam_dif_y # -cam offset then squish to range 0.0-1.0(0%-100%)
        botX = (camY * self.dobot_dif_x) + self.dobot_min_x # use cam (0%-100%) to map with dobot range, then add dobot offset
        return  botX
    
    def cam2botY(self,camX,camY):
        camX = (camX - self.cam_min_x) / self.cam_dif_x 
        botY = (camX * self.dobot_dif_y) + self.dobot_min_y
        return  botY

C2D_cal = C2D() # create object with class C2D

# cup location
def drop_cup(colorIndex):
    if colorIndex == 0:
        dobot.moveTo(235, -150, C2D_cal.dobot_max_z, -40) # Red
    elif colorIndex == 1:
        dobot.moveTo(300, -150, C2D_cal.dobot_max_z, -40) # Yellow
    elif colorIndex == 2:
        dobot.moveTo(365, -150, C2D_cal.dobot_max_z, -40) # Green
    elif colorIndex == 3:
        dobot.moveTo(275, -205, C2D_cal.dobot_max_z, -40) # Blue
    elif colorIndex == 4:
        dobot.moveTo(325, -205, C2D_cal.dobot_max_z, -40) # Purple
    else :
        print('wrong colorIndex variable')
        dobot.moveTo(300, 0, C2D_cal.dobot_max_z+50, -40) # some random spot if wrong colorIndex

In [ ]:
# Homing Dobot to close to base
dobot.moveTo(155, -12.6, 100, 40)

In [ ]:
##### Main 3 - Move Dobot ######
print('>> Phase 3')

# dobot.moveTo(x, y, z, r) # Absolute Position
# dobot.moveInc(dx, dy, dz, dr) # Relative Position
# dobot.setPump(17, 18) # Active Pump
# dobot.resetPump(17, 18) # Deactive Pump
# dobot.suck() # S U C K
# dobot.unsuck # un S U C K
# dobot.blow() # blow
# time.sleep(1) # sleep 1 sec

# dobot.moveInc(0, 0, 30, 0)
dobot.setPump(17, 18) # start pump (default status: blow)
dobot.suck() # no blow no suck --> tested

for i in object_list_merge_xy:
    print(f'Camera: x= {i[0]}, y= {i[1]}')
    x = C2D_cal.cam2botX(i[0],i[1]) # calculate x dobot from xy camera
    y = C2D_cal.cam2botY(i[0],i[1]) # calculate y dobot from xy camera
    color = i[2] # color index for drop
    print(f'Dobot: x= {x}, y= {y}, drop= {color}')

    dobot.moveTo(x, y, C2D_cal.dobot_max_z, -40) # to x,y
    time.sleep(1)
    dobot.unsuck() # suck --> tested

    dobot.moveTo(x, y, C2D_cal.dobot_min_z, -40) # lower z
    time.sleep(1)

    dobot.moveTo(x, y, C2D_cal.dobot_max_z, -40) # raise z
    time.sleep(1)

    drop_cup(color) # to above cup 
    time.sleep(1)
    dobot.blow() # blow --> tested

    time.sleep(0.1)
    dobot.suck() # idle pump

dobot.resetPump(17, 18) # disable? pump
dobot.moveTo(155, -12.6, 100, 40) # move Dobot to home? position

print('>> Exit Phase 3')